## 학습 파이프라인 디버깅 - 파이토치


In [1]:
# !pip install evaluate

  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)


- evalute: ai모델, 머신러닝, 데이터셋의 성능 평가 모듈
- glue: 언어를 이해하는 리소스 모음. 일반언어 이해평가, 자연어 이해 시스템의 훈련평가분류 리소스 모음. 
- mnli: 멀티장르 자연어추론 말뭉치

In [2]:
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install scikit-learn

     ---------------------------------------- 9.3/9.3 MB 28.3 MB/s eta 0:00:00
     ---------------------------------------- 302.0/302.0 kB ? eta 0:00:00


In [1]:
from datasets import load_dataset
import evaluate 
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

raw_datasets = load_dataset("glue", "mnli")

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) # 모델 체크포인트에 맞는 토크나이저를 이용한다.

# 데이터셋을 가져와서 토큰화
def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint) # 자동으로 시퀀스 분류 모델 아키텍처를 찾는다.

args = TrainingArguments(
    f"distilbert-finetuned-mnli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

metric = evaluate.load("glue", "mnli")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model,
    args,
    train_dataset=raw_datasets["train"],
    eval_dataset=raw_datasets["validation_matched"],
    compute_metrics=compute_metrics,
)
trainer.train()


Found cached dataset glue (C:/Users/user/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-cffe76e46a533d3e.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-68e72c5b90e42d40.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-3b10752cbf427225.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-dc2ceef30f561722.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-054ad03448e6fa1b.arrow
Some weights of the model checkpoint at distilbert-base-unca

  0%|          | 0/147264 [00:00<?, ?it/s]

ValueError: You have to specify either input_ids or inputs_embeds

- 'ValueError: You have to specify either input_ids or inputs_embeds'
1. 데이터 관련 에러가 생기면 trainer.train_dataset을 먼저 확인해야한다.

In [2]:
trainer.train_dataset[0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

- idx가 0인 걸 보면, 토큰이 수치화가 되지 않았음을 알 수 있다.
- 가장 하단 코드에서 train_dataset에 raw_datasets 대신 tokenized_datasets을 넣어준다.

In [2]:
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

raw_datasets = load_dataset("glue", "mnli")

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

args = TrainingArguments(
    f"distilbert-finetuned-mnli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

metric = evaluate.load("glue", "mnli")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"], # 이전 코드는 raw_datasets을 이용했다
    eval_dataset=tokenized_datasets["validation_matched"],
    compute_metrics=compute_metrics,
)
trainer.train()

Found cached dataset glue (C:/Users/user/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-cffe76e46a533d3e.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-68e72c5b90e42d40.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-3b10752cbf427225.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-dc2ceef30f561722.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-054ad03448e6fa1b.arrow
Some weights of the model checkpoint at distilbert-base-unca

  0%|          | 0/147264 [00:00<?, ?it/s]

ValueError: expected sequence of length 43 at dim 1 (got 37)

- 다른 오류 발생! 시퀀스의 길이가 43으로 맞춰줘야 한다.
- 디버깅을 할 때 항상 기억해야 될 것은 모델의 입력값을 디코딩해서 확인해봐야한다.
- 컴퓨터 비전에서는 픽셀의 디코딩된 그림, 음성에서는 디코딩된 오디오 샘플, NLP는 디코딩된 토큰봐야 한다.
- 토큰을 디코딩하기 위해서 토크나이저의 디코드를 이용한다.

In [6]:
trainer.train_dataset[0].keys()

dict_keys(['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'])

In [5]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

'[CLS] conceptually cream skimming has two basic dimensions - product and geography. [SEP] product and geography are what make cream skimming work. [SEP]'

- 아이디 리스트와 어텐션 마스크는 서로 길이가 같으니 패딩 문제는 아니다.

In [9]:
len(trainer.train_dataset[0]["attention_mask"]) == len(
    trainer.train_dataset[0]["input_ids"]
)

True

- 1은 neutural을 의미한다. 중립이라는 건데 첫 번째 문장이 두 번째 문장을 의미하지 않는다는 말이다. entailment = 수반, contradiction = 모순.
- 아무튼 데이터셋은 문제가 없다.

In [10]:
trainer.train_dataset[0]["label"]

1

In [11]:
trainer.train_dataset.features["label"].names

['entailment', 'neutral', 'contradiction']

- 학습 파이프라인을 확인해보자.
- 데이터셋이 문제가 없으면, Trainer가 배치를 형성할 때를 확인해보자.
- 아래 코드는 학습 데이터로더를 생성한 다음 반복하며 첫 번째 반복에서 중지한다.
- 에러가 없으면 첫 번째 배치를 형성하고, 에러가 있으면 배치를 형성하는 곳에서 문제가 생겼음을 알 수 있다.

In [13]:
for batch in trainer.get_train_dataloader():
    break

ValueError: expected sequence of length 45 at dim 1 (got 76)

- 파이토치의 Dataloader는 데이터셋을 모델에 학습을 시키기 위해, 배치 사이즈로 슬라이싱하는 역할이다. 데이터로더는 배치사이즈를 포함하여 여러 파라미터가 있는데, 그 중에 collate_fn이 있다.
- collate_fn은 인풋의 길이가 달라서 패딩을 추가할 때 사용한다.
- 보통 가장 긴 문장을 기준으로 길이가 작은 문장에 패딩을 추가한다.
- collate_fn이 작동하지 않은 이유는 Trainer에 tokenizer을 전달하지 않았기 때문에 DataCollaterWithPadding을 생성하지 못했다. 
- 명시적으로 콜래터 함수를 써주자.

In [14]:
data_collator = trainer.get_train_dataloader().collate_fn
data_collator

<function transformers.data.data_collator.default_data_collator(features: List[InputDataClass], return_tensors='pt') -> Dict[str, Any]>

In [2]:
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

raw_datasets = load_dataset("glue", "mnli")

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

args = TrainingArguments(
    f"distilbert-finetuned-mnli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

metric = evaluate.load("glue", "mnli")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions, references=labels)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # 명시적으로 사용

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    compute_metrics=compute_metrics,
    data_collator=data_collator, # 추가
    tokenizer=tokenizer,
)
trainer.train()

Found cached dataset glue (C:/Users/user/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-cffe76e46a533d3e.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-68e72c5b90e42d40.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-3b10752cbf427225.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-dc2ceef30f561722.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-054ad03448e6fa1b.arrow
Some weights of the model checkpoint at distilbert-base-unca

  0%|          | 0/147264 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


- 악명 높은 CUDA 오류라고 한다. CUDA 오류는 디버그하기가 매우 어렵다고 한다.
- 일단 data collator가 정상인지 확인해보자.
- Trainer._remove_unused_columns()메서드는 트레이너가 무대 뒤에서 수행하는 작업을 정확히 복제한다.

In [4]:
data_collator = trainer.get_train_dataloader().collate_fn
actual_train_set = trainer._remove_unused_columns(trainer.train_dataset)
batch = data_collator([actual_train_set[i] for i in range(4)])
batch

{'input_ids': tensor([[  101, 17158,  2135,  6949,  8301, 25057,  2038,  2048,  3937,  9646,
          1011,  4031,  1998, 10505,  1012,   102,  4031,  1998, 10505,  2024,
          2054,  2191,  6949,  8301, 25057,  2147,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2017,  2113,  2076,  1996,  2161,  1998,  1045,  3984,  2012,
          2012,  2115,  2504,  7910,  2017,  4558,  2068,  2000,  1996,  2279,
          2504,  2065,  2065,  2027,  5630,  2000,  9131,  1996,  1996,  6687,
          2136,  1996, 13980,  5630,  2000,  2655,  2000,  9131,  1037,  3124,
          2013,  6420,  1037,  2059,  1037,  3313, 

- 여전히 배치를 얻을 수 없다.

In [5]:
for batch in trainer.get_train_dataloader():
    break

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


- CUDA오류는 커널을 복구할 수 없을 정도로 망가뜨리고, 무엇보다 디버깅하기 어렵다.
- 디버깅 하기 어려운 이유는 GPU가 병렬로 실행되기 때문이다. 병렬로 실행이 되면 작업이 효율적이지만, 에러가 발생했을 때 즉시 알 수가 없다. Traceback을 보면 잘못된 방향으로 오류를 설명한다.
- 그래서! CPU로 돌아가 디버깅을 해야한다.

In [6]:
outputs = trainer.model.cpu()(**batch)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
'''
사실 이런 오류가 나와야 하는데... 계속 CUDA 오류가 나옴...

~/.pyenv/versions/3.7.9/envs/base/lib/python3.7/site-packages/torch/nn/functional.py in nll_loss(input, target, weight, size_average, ignore_index, reduce, reduction)
   2386         )
   2387     if dim == 2:
-> 2388         ret = torch._C._nn.nll_loss(input, target, weight, _Reduction.get_enum(reduction), ignore_index)
   2389     elif dim == 4:
   2390         ret = torch._C._nn.nll_loss2d(input, target, weight, _Reduction.get_enum(reduction), ignore_index)

IndexError: Target 2 is out of bounds.
'''

- 모델의 레이블 수를 확인하면 2다.
- 이전에 추출한 레이블 수는 3개다. 모델에게 레이블 3개가 생성되어야 하는 점을 알려주지 않았다.

In [7]:
trainer.model.config.num_labels

2

In [1]:
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

raw_datasets = load_dataset("glue", "mnli")

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3) # 레이블 수 3개야

args = TrainingArguments(
    f"distilbert-finetuned-mnli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

metric = evaluate.load("glue", "mnli")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions, references=labels)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Found cached dataset glue (C:/Users/user/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-cffe76e46a533d3e.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-68e72c5b90e42d40.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-3b10752cbf427225.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-dc2ceef30f561722.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-054ad03448e6fa1b.arrow
Some weights of the model checkpoint at distilbert-base-unca

- 문제가 없이 돌아간다.

### 최적화 단계

- 모델을 통과하는 배치를 만들었다.
- 학습 파이프라인 다음 단계인 경사하강과 최적화를 해보자.

In [2]:
data_collator = trainer.get_train_dataloader().collate_fn
actual_train_set = trainer._remove_unused_columns(trainer.train_dataset)
batch = data_collator([actual_train_set[i] for i in range(4)])

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [3]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: v.to(device) for k, v in batch.items()}
outputs = trainer.model.to(device)(**batch)

loss = outputs.loss
loss.backward()

In [4]:
trainer.create_optimizer()
trainer.optimizer.step()

c:\Users\user\miniconda3\envs\gpu2\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### 모델 평가

- 아무런 문제가 없어보이지만 마지막 디버깅 문제를 해결해야 한다.

In [5]:
device

device(type='cuda')

In [13]:
# 반드시 오류가 날 것이고, 오래 걸리니 돌리지 말자
# trainer.train()

  0%|          | 0/147264 [00:00<?, ?it/s]

{'loss': 0.8923, 'learning_rate': 1.9891487396784007e-05, 'epoch': 0.01}
{'loss': 0.8226, 'learning_rate': 1.9823582138200783e-05, 'epoch': 0.02}
{'loss': 0.7862, 'learning_rate': 1.975567687961756e-05, 'epoch': 0.03}
{'loss': 0.7788, 'learning_rate': 1.9687771621034334e-05, 'epoch': 0.04}
{'loss': 0.7582, 'learning_rate': 1.961986636245111e-05, 'epoch': 0.05}
{'loss': 0.7437, 'learning_rate': 1.9551961103867885e-05, 'epoch': 0.06}
{'loss': 0.7105, 'learning_rate': 1.948405584528466e-05, 'epoch': 0.07}
{'loss': 0.7069, 'learning_rate': 1.9416150586701435e-05, 'epoch': 0.08}
{'loss': 0.6676, 'learning_rate': 1.934824532811821e-05, 'epoch': 0.09}


KeyboardInterrupt: 

- 항상 trainer.train()을 실행하기 전에, trainer.evalute()를 실행하자. 컴퓨팅 리소스를 낭비를 방지할 수 있다.


In [6]:
for batch in trainer.get_eval_dataloader():
    break

batch = {k: v.to(device) for k, v in batch.items()}

with torch.no_grad():
    outputs = trainer.model(**batch)

- 평가 루프에서 문제를 디버깅하기 전에 데이터를 잘 살펴봤는지, 배치를 적절하게 구성할 수 있는지, 모델을 실행할 수 있는지 확인해야 한다.

In [9]:
trainer.evaluate()

ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)},
Input predictions: [[-0.00070621 -0.075327    0.09945462]
 [ 0.04038714 -0.04707627  0.06016049]
 [ 0.02944145 -0.08373697  0.05960576]
 ...
 [-0.00641662  0.00904457  0.12480208]
 [ 0.01996658 -0.06131065  0.05024731]
 [ 0.00848687 -0.03649977  0.06220504]],
Input references: [1 2 0 ... 0 0 2]

In [7]:
predictions = outputs.logits.cpu().numpy()
labels = batch["labels"].cpu().numpy()

compute_metrics((predictions, labels))

ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)},
Input predictions: [[ 0.1123177   0.00854467  0.04766916]
 [ 0.07452206  0.03631864  0.02012929]
 [ 0.09268822 -0.03561747 -0.01512304]
 [ 0.09779865 -0.04101551  0.019991  ]
 [ 0.11367577 -0.00187646  0.02853778]
 [ 0.08824772 -0.03343624 -0.03619592]
 [ 0.06390588 -0.02718233 -0.01299236]
 [ 0.03369591 -0.0201085  -0.00244073]],
Input references: [1 2 0 2 2 2 2 1]

```
Cell In[1], line 38, in compute_metrics(eval_pred)
     36 def compute_metrics(eval_pred):
     37     predictions, labels = eval_pred
---> 38     return metric.compute(predictions=predictions, references=labels)
```
- compute_metrics가 문제인 것을 알 수 있다.

In [ ]:
# 여전히 로짓이다. 실제 예측값이 아니다.
predictions.shape, labels.shape

((8, 3), (8,))

- compute_metrics() 함수에 argmax를 추가해주면 된다.

In [9]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)


compute_metrics((predictions, labels))

{'accuracy': 0.125}

In [11]:
import numpy as np
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

raw_datasets = load_dataset("glue", "mnli")

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

args = TrainingArguments(
    f"distilbert-finetuned-mnli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

metric = evaluate.load("glue", "mnli")

# 오버라이트
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
# trainer.train()

Found cached dataset glue (C:/Users/user/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-cffe76e46a533d3e.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-68e72c5b90e42d40.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-3b10752cbf427225.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-dc2ceef30f561722.arrow
Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-054ad03448e6fa1b.arrow
Some weights of the model checkpoint at distilbert-base-unca

### 과연 학습이 잘됐을까?
- 에러 없이 학습이 완료됐지만 좋은 결과르 얻지 못하는 경우가 있다.
- 이런 학습을 디버깅하려면 어떻게 할까?

#### 스스로 물어볼만한 질문
- 디코딩된 데이터를 이해할만 한지?
- 예측된 레이블에 납득이 가는지?
- 다른 레이블보다 정답에 가까운 레이블이 있는지?
- 모델이 무작위 답, 언제나 같은 답을 예측할 경우 어떤 손실함수와 매트릭을 정해야 할 지?

#### 배치 한번으로 모델의 과적합 해보기

- 모델은 데이터에서 실제로 뭔가 학습할 수 있는 경우에만 학습한다.
- 실제로 잘 학습이 되고 있는지를 확인하려면 하나의 배치를 반복해서 과적합으로 만들어 보는 방법이 있다.
- 과적합은 훈련 샘플을 암기하는 것과 마찬가지여서, 새로운 데이터를 예측하지 못한다.
- 하지만 반대로 생각해보면 학습이 잘되고 있다는 말이다.
- 모델에서 설정한 문제를 해결할 수 있는지 확인하는 좋은 테스트다.


In [12]:
# 배치 한번만 받아오기
for batch in trainer.get_train_dataloader(): 
    break

# 아이디 리스트, 어텐션 마스크, 레이블
batch = {k: v.to(device) for k, v in batch.items()}
# 트레이너에 맞는 옵티마이저 인스턴스를 생성한다.
trainer.create_optimizer()

for _ in range(20):
    outputs = trainer.model(**batch) # 딕셔너리 형태를 언팩한다.
    loss = outputs.loss # 손실, 출력이 정답으로부터 얼마나 떨어져 있는지를 계산.
    loss.backward() # 미분값 생성, 역전파를 수행
    trainer.optimizer.step() # 파라미터에 옵티마이저 적용
    trainer.optimizer.zero_grad() # 배치마다 다른 미분값을 사용하기 위해 이전 이터레이션의 미분을 초기화

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\user\miniconda3\envs\gpu2\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


- 만약 모델의 정확도가 100%로 되지 않는다면 문제 또는 데이터를 구성하는 방식에 문제가 있음을 의미한다.
- 과적합 테스트를 통과해야만 모델이 실제로 무언가를 배울 수 있다는 것을 확신할 수 있다.

In [20]:
with torch.no_grad():
    outputs = trainer.model(**batch)
preds = outputs.logits
labels = batch["labels"]

compute_metrics((preds.cpu().numpy(), labels.cpu().numpy()))

{'accuracy': 1.0}

#### 첫 기준 모델을 만들기전엔 튜닝하지 마세요.
- 하이퍼파라미터 튜닝은 메트릭에 대해 약간의 정보를 얻는 데 도움이 되는 마지막 단게다.
- 대부분의 경우 기본 하이퍼파라미터가 잘 작동하여 좋은 결과를 얻을 수 있다.
- 그러니 하이퍼파라미터를 찾을 때까지 시간과 비용을 많이 들이지 말자.
<br><br>
- 적합한 모델이 있으면 약간의 튜닝을 시작할 수 있다.
- 서로 다른 하이퍼파라미터를 수천 번 실행하지 말자.
- 하나의 하이퍼파라미터에 대해 서로 다른 값을 가진 두번의 실행을 비교하여 가장 큰 영향을 미치는 아이디어를 얻어보자.

#### 도움이 될만한 자료
- [“Reproducibility as a vehicle for engineering best practices” by Joel Grus](https://docs.google.com/presentation/d/1yHLPvPhUs2KGI5ZWo0sU-PKU3GimAk3iTsI38Z-B5Gw/edit#slide=id.p)
- [“Checklist for debugging neural networks” by Cecelia Shao](https://towardsdatascience.com/checklist-for-debugging-neural-networks-d8b2a9434f21)
- [“How to unit test machine learning code” by Chase Roberts](https://thenerdstation.medium.com/how-to-unit-test-machine-learning-code-57cf6fd81765)
- [“A Recipe for Training Neural Networks” by Andrej Karpathy](http://karpathy.github.io/2019/04/25/recipe/)

---